In [4]:
# lookup index in sorted values
import bisect
import pathlib

# io
import netCDF4
# siggyf/pyugrid#develop branch (waiting to be merged into pyugrid pr#144)
import pyugrid
import rasterio
import rasterio.plot
import rasterio.windows

# computing
import numpy as np
import shapely.geometry
import pandas

# notebook extensions
from ipywidgets import interact, interactive, fixed
import tqdm

# plotting
import cmocean.cm
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.style
# matplotlib.style.use('ggplot')

import flowmap.subgrid
import flowmap.formats
import flowmap.dem

%matplotlib inline

In [5]:
# tiff file that has a correction for the 1d bathymetry (deburned channels) 
# original bathymetry with burned streams is named 'dem_3Di_2272015.tif'
dem_filename = pathlib.Path('D:/vries_cy/Desktop/Groesbeek/aw_ahn_d_asc.tiff').expanduser()

fm_filename = pathlib.Path('D:/vries_cy/Desktop/Groesbeek/groesbeek_map.nc').expanduser()
# fm_filename = pathlib.Path('D:/vries_cy/Desktop/Groesbeek refined/groesbeek_map.nc').expanduser()

t = -1

This notebook gives an example of how to apply a subgrid technique to an arbitrary numerical model result. 
A flow field is computed using a course schematization. In this example the coarse model computes on a 8m x 8m resolution. The dem is available at 0.5m resolution. So the question is, how do we take into account the high resolution dem after the model has already determined the volumes and fluxes in the coarse grid cells. 

We take the approach of redistrbuting the volumes by filling up the detailed bathymetry from the bottom.
Other approaches are to fill the bathymetry using an interpolated function or from the cell edge with the largest volume difference. 

Advantages of this method over other methods:
- Only computed for flooded areas
- Can be applied a posteriori. No need to implement subgridding in the model.
- Flexibility in detail (only compute on visualization)


Dem
====
Read the high detailed dem file

In [6]:

dem = flowmap.dem.read_dem(dem_filename)


Model
=====

Read the model output

In [7]:
ugrid = flowmap.formats.UGrid(fm_filename)

Example
======

In [8]:
# read the dem

data = ugrid.waterlevel(-1)

In [16]:
x, y = 1000, 1000
delta = 1000
s = np.s_[(y-delta):(y+delta), (x-delta):(x+delta)]

In [22]:
 
import pandas as pd
def build_tables(grid, dem):
    """compute volume tables per cell"""

    # compute cache of histograms per cell
    idx = range(grid['face_coordinates'].shape[0])
    faces = grid['face_coordinates'][idx]
    tables = {}
    for id_, face in zip(idx, tqdm.tqdm(faces)):
        affine = dem['affine']
        face_px = dem['world2px'](face)
        face_px2slice = np.s_[
            face_px[:, 1].min():face_px[:, 1].max(),
            face_px[:, 0].min():face_px[:, 0].max()
        ]
        dem_i = dem['band'][face_px2slice]
        if not dem_i.mask.all():
            n, bins = np.histogram(dem_i.ravel(), bins=20)
            n_cum = np.cumsum(n)
            volume_table = np.abs(affine.a * affine.e) * n_cum * np.diff(bins)
            cum_volume_table = np.cumsum(volume_table)
        else:
            n, bins = None, None
            volume_table = None,
            cum_volume_table = None
        extent = [
            face[:, 0].min(),
            face[:, 0].max(),
            face[:, 1].min(),
            face[:, 1].max()
        ]
        record = dict(
            id=id_,
            slice=face_px2slice,
            face=face,
            dem=dem_i,
            volume_table=volume_table,
            cum_volume_table=cum_volume_table,
            n=n,
            extent=extent,
            bins=bins
        )
        tables[id_] = record

    tables = pd.DataFrame.from_records(list(tables.values())).set_index('id')
    return tables
tables = build_tables(ugrid.ugrid, dem)


100%|████████████████████████████████▉| 708746/708777 [12:57<00:00, 911.00it/s]

In [70]:
def compute_subgrid_waterdepth(grid, dem, tables, data, mode='mean'):
    """compute subgrid waterdepth band"""
    excluded = []
    faces = list(tables.index)
    print(data['waterdepth'][0])
    mean_waterdepth = np.zeros_like(data['waterdepth'])
    median_waterdepth = np.zeros_like(data['waterdepth'])
    low_waterdepth = np.zeros_like(data['waterdepth'])

#     # create a masked array
#     band = np.ma.masked_all_like(dem['band'])

    # fill the in memory band
    for i, face_idx in enumerate(tqdm.tqdm_notebook(faces)):
        row = tables.loc[face_idx]
        waterdepth_i = (flowmap.subgrid.subgrid_waterdepth(face_idx, 
                                                                 dem=dem, 
                                                                 grid=grid, 
                                                                 data=data, 
                                                                 tables=tables))
        if waterdepth_i is None:
            mean_waterdepth[i] =  None
            median_waterdepth[i] = None
            low_waterdepth[i] = None
        else:
            mean_waterdepth[i] = np.mean(waterdepth_i)
            median_waterdepth[i] = np.median(waterdepth_i)
            low_waterdepth[i] = waterdepth_i.min()
#             if mode =='mean':
#                 new_waterdepth[i] = np.mean(waterdepth_i)
#             elif mode == 'median':
#                 new_waterdepth[i] = np.median(waterdepth_i)
#             elif mode == 'lowest':
#                 new_waterdepth[i] = waterdepth_i.min()


    return mean_waterdepth, median_watedepth, low_waterdepth
mean_waterdepth, median_watedepth, low_waterdepth = compute_subgrid_waterdepth(grid, dem, tables, data, mode='mean')

In [47]:
band = flowmap.subgrid.compute_band(ugrid.ugrid, dem, tables, data)

HBox(children=(IntProgress(value=0, max=708777), HTML(value='')))

Compute volume interpolation


In [31]:
# mean_waterdepth = compute_subgrid_waterdepth(ugrid.ugrid, dem, tables, data, mode='mean')

0.00974997878075
0.00514985686657


HBox(children=(IntProgress(value=0, max=708777), HTML(value='')))

In [91]:
# median_waterdepth = compute_subgrid_waterdepth(ugrid.ugrid, dem, tables, data, mode='median')

In [71]:
# low_waterdepth = compute_subgrid_waterdepth(ugrid.ugrid, dem, tables, data, mode='lowest')

0.00514985686657


HBox(children=(IntProgress(value=0, max=708777), HTML(value='')))

Exception in thread Thread-23:
Traceback (most recent call last):
  File "C:\Anaconda\envs\py36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Anaconda\envs\py36\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Anaconda\envs\py36\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



In [32]:
is_grid_full = ugrid.build_is_grid(raster=dem)


0it [00:00, ?it/s]
1it [00:00,  6.47it/s]
644it [00:00, 2540.44it/s]
1284it [00:00, 3622.00it/s]
1928it [00:00, 4232.71it/s]
2441it [00:00, 4398.20it/s]
3062it [00:00, 4674.81it/s]
3731it [00:00, 4938.45it/s]
4364it [00:00, 5101.11it/s]
4939it [00:00, 5158.23it/s]
5578it [00:01, 5274.71it/s]
6170it [00:01, 5312.10it/s]
6817it [00:01, 5406.03it/s]
7422it [00:01, 5445.34it/s]
Exception in thread Thread-15:
Traceback (most recent call last):
  File "C:\Anaconda\envs\py36\lib\threading.py", line 916, in _bootstrap_inner
    self.run()
  File "C:\Anaconda\envs\py36\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Anaconda\envs\py36\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

708777it [01:44, 6753.79it/s]


In [39]:
values = np.c_[data['s1'], data['vol1'], data['waterdepth']]
L = flowmap.subgrid.build_interpolate(ugrid.ugrid, values)

In [40]:
values = np.c_[data['s1'], data['vol1'], mean_waterdepth]
L_SI = flowmap.subgrid.build_interpolate(ugrid.ugrid, values)

In [59]:
values = np.c_[data['s1'], data['vol1'], median_waterdepth]
L_SI_median = flowmap.subgrid.build_interpolate(ugrid.ugrid, values)

In [73]:
values = np.c_[data['s1'], data['vol1'], low_waterdepth]
L_SI_low = flowmap.subgrid.build_interpolate(ugrid.ugrid, values)

In [85]:
def compute_interpolated(L, dem, data, wdepth, s=None):
    """compute a map of interpolated waterdepth, masked where detailed topography >= interpolated waterlevel, optionally sliced by a tuple (s) of row, column slices"""
    if s is None:
        s = np.s_[:, :]

    # create the pixel grid (assuming no rotation)
    affine = dem['affine']
    assert affine.b == 0 and affine.d == 0, 'rotated dems not implemented'
    y = np.arange(affine.f, affine.f + affine.e * dem['height'], affine.e)
    x = np.arange(affine.c, affine.c + affine.a * dem['width'], affine.a)
    # we need the full grid to get the interpolated values
    X, Y = np.meshgrid(x[s[1]], y[s[0]])
    # fill the interpolation function
    msg = 'Interpolation function should be filled with s1, vol1, and waterdepth'
    assert L.values.shape[1] == 3, msg
    # fill in new values
    L.values = np.c_[data['s1'], data['vol1'], wdepth]
    # compute interplation
    interpolated = L(X, Y)
    # get the variables
    s1 = interpolated[..., 0]
    waterdepth = interpolated[..., 2]
    vol1 = interpolated[..., 1]
#     new_waterdepth = interpolated[..., 3]
    # lookup band
    dem_band = dem['band'][s]
    # mask interpolated values using dem
    masked_waterdepth = np.ma.masked_array(waterdepth, mask=dem_band >= s1)
    return {
        "masked_waterdepth": masked_waterdepth,
        "s1": s1,
        "vol1": vol1,
        "dem": dem_band, 
    }


In [90]:
delta = 100

@interactive 
def plot(x=(2*delta,dem['width'], 2*delta), y=(2*delta, dem['width'], 2*delta)):
    # row columns
    s = np.s_[(y-delta):(y+delta), (x-delta):(x+delta)]
    
    interpolated = compute_interpolated(L, dem, data, data['waterdepth'], s=s)
    masked_waterdepth = interpolated['masked_waterdepth']
    interpolated_SI = compute_interpolated(L_SI, dem, data, mean_waterdepth, s=s)
    masked_waterdepth_SI = interpolated['masked_waterdepth']
    interpolated_SI_median = compute_interpolated(L_SI_median, dem, data, median_waterdepth, s=s)
    masked_waterdepth_SI_median = interpolated['masked_waterdepth']   
    interpolated_SI_low = compute_interpolated(L_SI_low, dem, data, low_waterdepth, s=s)
    masked_waterdepth_SI_low = interpolated['masked_waterdepth']
#     is_grid = is_grid_full[s]
    s1 = interpolated['s1']    
    vol1 = interpolated['vol1']
    dem_band = interpolated['dem']
    
    
    fig, axes = plt.subplots(2, 3, figsize=(20, 10))
    im = axes[0, 0].imshow(s1 - dem_band, cmap='Blues')
    plt.colorbar(im, ax=axes[0, 0])
#     axes[0, 0].set_title('s1 - dem')
#     im = axes[0, 1].imshow(s1, cmap='viridis')
#     plt.colorbar(im, ax=axes[0, 1])
#     axes[0, 1].set_title('s1')
#     im = axes[0, 2].imshow(dem_band, cmap='viridis')
#     plt.colorbar(im, ax=axes[0, 2])
#     axes[0, 2].set_title('dem')
#     im = axes[1, 0].imshow(vol1, cmap='viridis', vmax=20)
#     plt.colorbar(im, ax=axes[1, 0])
#     axes[1, 0].set_title('vol1')
    vmax = 0.05

    im = axes[0, 1].imshow(masked_waterdepth_SI, cmap='Blues', vmax=vmax)
    plt.colorbar(im, ax=axes[0, 1])
    axes[0, 1].set_title('mean waterdepth subgrid interpolated')
    
    im = axes[1, 1].imshow(masked_waterdepth, cmap='Blues', vmax=vmax)
    plt.colorbar(im, ax=axes[1, 1])
    axes[1, 1].set_title('waterdepth (interpolated and masked)')

    im = axes[1, 0].imshow(band[s], cmap='Blues', vmax=vmax)
    plt.colorbar(im, ax=axes[1, 0])
    axes[1, 0].set_title('waterdepth subgrid')
    
    im = axes[0, 2].imshow(masked_waterdepth_SI_median, cmap='Blues', vmax=vmax)
    plt.colorbar(im, ax=axes[0, 2])
    axes[0, 2].set_title('median waterdepth subgrid interpolated')

    im = axes[1, 2].imshow(masked_waterdepth_SI_low, cmap='Blues', vmax=vmax)
    plt.colorbar(im, ax=axes[1, 2])
    axes[1, 2].set_title('low waterdepth subgrid interpolated')
plot

interactive(children=(IntSlider(value=8000, description='x', max=16096, min=200, step=200), IntSlider(value=8000, description='y', max=16096, min=200, step=200), Output()), _dom_classes=('widget-interact',))